In [33]:
import torch
import torchvision
import torchvision.models as models
from torchvision import transforms
from torchvision.models.resnet import ResNet50_Weights
import lightning.pytorch as pl
from ultralytics import YOLO

from mymodels import Model_Wrapper, Preprocess
from myutils import View, sample_imgs_list, compare_ds

In [35]:
# configs to speed up training
torch.set_float32_matmul_precision('medium')
torch.backends.cuda.matmul.allow_tf32 = True
torch.backends.cudnn.allow_tf32 = True

In [36]:
model = YOLO('yolov8s.pt', )  # get pretrained model
model = Model_Wrapper(model, type='yolo')  # wrap model to use with lightning
trainer = pl.Trainer(accelerator='auto', limit_test_batches=3)

In [37]:
PATH_TO_COCO = '/f_storage/datasets/coco/2017/val2017'
PATH_TO_IMAGENET = '../../datasets/imagenet/2012/'
NUM_IMG_EVAL = 10000
prep = Preprocess(PATH_TO_COCO, (640, 640), dataset_type='coco', shuffle=False, batch_size=8)
preview_img_slice = slice(00000, 50000, 10000)
%matplotlib inline

In [41]:
prep.reset_trans()
imgnet = prep.get_loader()
img = sample_imgs_list(imgnet, slice(31000,32000))


In [ ]:
prep.reset_trans()
imgnet_orig = prep.get_loader()
sample_orig = sample_imgs_list(imgnet_orig, preview_img_slice)# test model on original images
result_orig = trainer.test(model, imgnet_orig)